In [1]:
from parse_deck import parse_events_from_official

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import re
from tqdm import tqdm

import pandas as pd
from collections import OrderedDict

import json
import os

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def wait_loading_circle(timeout: int = 20):
    WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='sk-circle-container']")))
    

In [4]:
# decks = {
#     deck category 1: [
#         {
#             deck_link: "",
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             num_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }
all_categories = {
    "simple": ["ルギアVSTAR", "ミュウVMAX", "ムゲンダイナVMAX", "キュレムVMAX"],  # if this card exists, then we could decide the category
    "complicate": ["ゾロア", "レジドラゴVSTAR", "ギラティナVSTAR"]
}

decks = {}
store_file_name = 'store.json'
if os.path.exists(store_file_name):
    with open(store_file_name, 'r') as f:
        decks = json.load(f)

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])

In [5]:
len(store_code_list)

607

In [6]:
# parse events and save to json
parse_events_from_official(decks, all_categories, store_code_list, page_limit=30, event_limit=1000)

with open(store_file_name, 'w') as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

 50%|█████████████████████▌                     | 10/20 [00:26<00:50,  5.05s/it]

Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: headless chrome=108.0.5359.94)
Stacktrace:
0   chromedriver                        0x000000010ae53f38 chromedriver + 4910904
1   chromedriver                        0x000000010add3a03 chromedriver + 4385283
2   chromedriver                        0x000000010aa18747 chromedriver + 472903
3   chromedriver                        0x000000010aa5d34c chromedriver + 754508
4   chromedriver                        0x000000010aa5d5a1 chromedriver + 755105
5   chromedriver                        0x000000010aa4f216 chromedriver + 696854
6   chromedriver                        0x000000010aa8313d chromedriver + 909629
7   chromedriver                        0x000000010aa4f0ed chromedriver + 696557
8   chromedriver                        0x000000010aa832ce chromedriver + 910030
9   chromedriver                        0x000000010aa9e28e chromedriver + 1020558
10  chromedriver                      

100%|███████████████████████████████████████████| 20/20 [03:27<00:00, 10.36s/it]


In [7]:
# modified categories
decks.keys()

dict_keys(['others', 'ルギアVSTAR', 'ミュウVMAX', 'LOST ギラティナVSTAR', 'レジドラゴVSTAR', 'アルレジドラゴVSTAR', 'ゾロア', 'Other ギラティナVSTAR', 'ムゲンダイナVMAX', 'キュレムVMAX'])

In [8]:
target_category = "ゾロア"
df_list = []
common_cols = ["date", "num_people", "rank"]

start_date = "2022年11月12日"  # include
end_date = "2022年12月09日"  # include

for card_type in ["pokemons", "tools", "supporters", "stages", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[target_category]):
        deck_link = deck["deck_link"]  # row id
        if deck["date"] < start_date or deck["date"] > end_date:
            continue
        pokecard = OrderedDict()
        pokecard["date"] = deck["date"]
        pokecard["num_people"] = deck["num_people"]
        pokecard["rank"] = deck["rank"]
        pokecard.update(deck[card_type])
        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_link])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_link])])
    df = df.fillna(0)
    
    # sort rows by date
    df = df.sort_values(by=['date'], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols: col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis='rows', numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis='rows')
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks
    
    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows-1, num_rows-2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]
    
    # sort cols by pick rate
    df = df.sort_values('pick_rate', axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols: col_list.remove(c)
    df = df[common_cols + col_list]
    
    # store
    df_list.append(df)

In [9]:
len(df_list[0]) - 2

23

In [10]:
df_list[0].to_csv(f"{target_category}-pokecard-{start_date}-{end_date}.csv")
df_list[0]

,date,num_people,rank,ゾロアーク,ゾロア,タルップル,マナフィ,ライチュウ,グラエナ,ヤドラン,チラーミィ,チラチーノ,テールナー,バサギリ,アップリュー,かがやくジラーチ,ヒスイ ゾロアーク,ミノマダム,ピカチュウ,ビーダル,ビッパ,チルタリス,チルット,かがやくルチャブル
pick_rate,NaN,NaN,NaN,1.0,1.0,1.0,1.000000,1.000000,1.0,1.0,0.956522,0.956522,0.956522,0.869565,0.73913,0.521739,0.304348,0.217391,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
avg_num_used,NaN,NaN,NaN,4.0,4.0,1.0,1.130435,1.043478,1.0,1.0,3.272727,2.909091,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SRMyyy-RUAigw-RUpMyy,2022年12月07日(水),32.0,5.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,3.000000,3.000000,1.000000,1.000000,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,3.000000,3.000000,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,4.000000,3.000000,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,4.000000,3.000000,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,3.000000,3.000000,1.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,3.000000,2.000000,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,3.000000,3.000000,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.0,4.0,1.0,1.000000,1.000000,1.0,1.0,4.000000,3.000000,1.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
df_list[1].to_csv(f"{target_category}-tool-{start_date}-{end_date}.csv")
df_list[1]

,date,num_people,rank,レベルボール,しんかのおこう,ふつうのつりざお,レスキューキャリー,ハイパーボール,クイックボール,やまびこホーン,こだわりベルト,バトルVIPパス,ヒスイのヘビーボール,ともだちてちょう,ロストスイーパー,ふうせん,あなぬけのヒモ
pick_rate,NaN,NaN,NaN,1.000000,1.000000,1.000000,0.913043,0.608696,0.521739,0.434783,0.391304,0.26087,0.217391,0.173913,0.173913,0.130435,0.086957
avg_num_used,NaN,NaN,NaN,3.956522,3.434783,1.956522,1.619048,2.428571,2.333333,1.000000,1.111111,4.00000,1.000000,1.000000,1.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SRMyyy-RUAigw-RUpMyy,2022年12月07日(水),32.0,5.0,4.000000,4.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000,0.000000,1.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.000000,3.000000,2.000000,2.000000,0.000000,3.000000,0.000000,1.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.000000,4.000000,3.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.000000,4.000000,3.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.000000,3.000000,2.000000,1.000000,3.000000,3.000000,1.000000,1.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.000000,4.000000,2.000000,2.000000,4.000000,0.000000,1.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.000000,4.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.000000,4.000000,2.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
df_list[2].to_csv(f"{target_category}-supporter-{start_date}-{end_date}.csv")
df_list[2]

,date,num_people,rank,セレナ,博士の研究,ボスの指令,マリィ,シロナの覇気,クララ,さぎょういん,ユウリ,とりつかい,ハマナのバックアップ,ザクロ,カトレア,ヒガナの決意,オニオン,ネズ,シバ,ツツジ
pick_rate,NaN,NaN,NaN,1.000000,0.956522,0.739130,0.739130,0.130435,0.130435,0.086957,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478
avg_num_used,NaN,NaN,NaN,3.956522,2.818182,1.058824,2.117647,1.333333,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SRMyyy-RUAigw-RUpMyy,2022年12月07日(水),32.0,5.0,4.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.000000,3.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.000000,2.000000,0.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.000000,3.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [17]:
df_list[3].to_csv(f"{target_category}-stage-{start_date}-{end_date}.csv")
df_list[3]

,date,num_people,rank,頂への雪道,ポケストップ,未開の祭壇
pick_rate,NaN,NaN,NaN,0.782609,0.086957,0.043478
avg_num_used,NaN,NaN,NaN,1.833333,1.500000,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/SRMyyy-RUAigw-RUpMyy,2022年12月07日(水),32.0,5.0,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,1.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,2.000000,0.000000,0.000000


In [18]:
df_list[4].to_csv(f"{target_category}-energy-{start_date}-{end_date}.csv")
df_list[4]

,date,num_people,rank,ツインエネルギー,ダブルターボエネルギー,キャプチャーエネルギー,基本超エネルギー,ギフトエネルギー
pick_rate,NaN,NaN,NaN,1.0,0.956522,0.956522,0.086957,0.086957
avg_num_used,NaN,NaN,NaN,4.0,2.090909,2.909091,2.000000,2.500000
https://www.pokemon-card.com/deck/confirm.html/deckID/SRMyyy-RUAigw-RUpMyy,2022年12月07日(水),32.0,5.0,4.0,0.000000,2.000000,0.000000,4.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.0,3.000000,2.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.0,3.000000,3.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.0,3.000000,3.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.0,2.000000,3.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.0,2.000000,4.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.0,1.000000,3.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.0,1.000000,4.000000,0.000000,0.000000


In [19]:
len(decks["others"])

741